In [ ]:
import numpy as np
import pandas as pd
import librosa
import os


In [ ]:
def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1, track_id=1)

    columns = []
    for name, size in feature_sizes.items():
        it = ((name, '{:02d}'.format(i + 1)) for i in range(size))
        columns.extend(it)

    names = ('feature', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()

In [ ]:
def compute_features(tid, filepath):
    features = pd.Series(index=columns(), dtype=np.float32, name=tid)



    def feature_stats(name, values):
        features[name] = np.mean(values, axis=1)
        
    x, sr = librosa.load(filepath)  # kaiser_fast

    f = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
    feature_stats('zcr', f)

    cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12,
                             n_bins=7 * 12, tuning=None))
    assert cqt.shape[0] == 7 * 12
    assert np.ceil(len(x) / 512) <= cqt.shape[1] <= np.ceil(len(x) / 512) + 1

    f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
    feature_stats('chroma_cqt', f)
    f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
    feature_stats('chroma_cens', f)
    f = librosa.feature.tonnetz(chroma=f)
    feature_stats('tonnetz', f)
    features['track_id'] = track_id

    del cqt
    stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
    assert stft.shape[0] == 1 + 2048 // 2
    assert np.ceil(len(x) / 512) <= stft.shape[1] <= np.ceil(len(x) / 512) + 1
    del x

    f = librosa.feature.chroma_stft(S=stft ** 2, n_chroma=12)
    feature_stats('chroma_stft', f)

    f = librosa.feature.rmse(S=stft)
    feature_stats('rmse', f)

    f = librosa.feature.spectral_centroid(S=stft)
    feature_stats('spectral_centroid', f)
    f = librosa.feature.spectral_bandwidth(S=stft)
    feature_stats('spectral_bandwidth', f)
    f = librosa.feature.spectral_contrast(S=stft, n_bands=6)
    feature_stats('spectral_contrast', f)
    f = librosa.feature.spectral_rolloff(S=stft)
    feature_stats('spectral_rolloff', f)

    mel = librosa.feature.melspectrogram(sr=sr, S=stft ** 2)
    del stft
    f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
    feature_stats('mfcc', f)

    return features

In [ ]:
root_path = '/Users/saurabh/PycharmProjects/IDS/final_project/music-genre-classification/data/2018655052_small'
tracks_dict = {}
for path, subdirs, files in os.walk(root_path):
    for name in files:
        if name.split('.')[0].isdigit():
            tracks_dict[name.split('.')[0]] = os.path.join(path, name)

In [ ]:
total_features = {}
df = pd.DataFrame(columns=columns())
count = 0
error_dict = {}
for track_id, filepath in tracks_dict.items():
    try:
        features = compute_features(track_id, filepath)
        df = df.append(features, ignore_index=True)
    except:
        error_dict[track_id] = filepath
print(error_dict)
df.to_csv('./data/features.csv')